In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_XMM-13hr/data/holes_XMM-13hr_irac1_O16_20190201_WARNING-MADE-WITH-Lockman-SWIRE-PARAMS.fits')

## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../../dmu28/dmu28_XMM-13hr/data/XMM-13r_Ldust_prediction_results.fits')

In [5]:
cigale['id'].name = 'help_id'

In [6]:
cigale

help_id,bayes.stellar.m_star,bayes.stellar.m_star_err,bayes.sfh.sfr10Myrs,bayes.sfh.sfr10Myrs_err,bayes.dust.luminosity,bayes.dust.luminosity_err,best.chi_square,best.reduced_chi_square,best.attenuation.Av_BC,best.attenuation.BC_to_ISM_factor,best.attenuation.bessell_b,best.attenuation.galex_fuv,best.attenuation.slope_BC,best.attenuation.slope_ISM,best.attenuation.stellar.old,best.attenuation.stellar.young,best.dust.alpha,best.dust.gamma,best.dust.luminosity,best.dust.mass,best.dust.qpah,best.dust.umin,best.lyc_absorption_f_dust,best.lyc_absorption_f_esc,best.sfh.age,best.sfh.burst_age,best.sfh.f_burst,best.sfh.integrated,best.sfh.sfr,best.sfh.sfr100Myrs,best.sfh.sfr10Myrs,best.sfh.tau_burst,best.sfh.tau_main,best.stellar.age_m_star,best.stellar.imf,best.stellar.lum_ly,best.stellar.lum_ly_old,best.stellar.lum_ly_young,best.stellar.m_gas,best.stellar.m_gas_old,best.stellar.m_gas_young,best.stellar.m_star,best.stellar.m_star_old,best.stellar.m_star_young,best.stellar.metallicity,best.stellar.n_ly,best.stellar.n_ly_old,best.stellar.n_ly_young,best.stellar.old_young_separation_age,best.universe.age,best.universe.luminosity_distance,best.universe.redshift,best.90prime_g,best.90prime_r,best.mosaic_z,best.ukidss_j
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mJy,mJy,mJy,mJy
bytes27,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J133202.919+375129.897,61888472620.61961,21725005143.31443,281.02427824935876,541.3660837489078,6.7753380691681886e+38,1.0561392749731938e+39,0.6731954964177762,0.2243984988059254,3.8,0.8,3.8187706131949595,8.482669152059293,-0.7,-0.7,1.3391778529582707e+38,1.1890397496593217e+38,2.0,0.02,2.5282176026175924e+38,1.3804001200454077e+39,0.47,5.0,0.0,0.0,2500.0,10.0,0.001,87936006458.92775,60.983061441238874,52.88657979552166,60.971206469715675,10000.0,3000.0,907.548715023767,1.0,2.4801854324209564e+37,1.301171695104108e+35,2.4671737154699156e+37,32667062811.376884,32627456993.070683,39605818.30620326,55268943236.304726,54698837401.15992,570105835.1447982,0.02,7.594254158195595e+54,3.8999245726241545e+52,7.555254912469354e+54,10.0,9629.221086271891,6.321941668704366e+25,0.38,0.0028700829712951918,0.011823157007606364,0.029185027005963163,0.05588057095725763
HELP_J133203.944+375126.449,123406814620.76628,35028790723.63045,331.23711027003964,722.0603574142683,8.421040727225891e+38,1.4236116737437414e+39,5.396845718404212,1.7989485728014039,3.3,0.5,2.2190075945143195,5.563379238278754,-0.7,-0.7,7.863583919389639e+37,9.00680977767177e+37,2.0,0.02,1.6870393697061407e+38,9.211190310725514e+38,0.47,5.0,0.0,0.0,8000.0,10.0,0.001,253191287780.72153,46.27601634688778,23.717363506889072,46.30707301681012,10000.0,3000.0,3862.5078086081307,1.0,1.888472633100084e+37,1.5784613461425643e+35,1.8726880196386583e+37,112423750722.82846,112393659015.54762,30091707.280843034,140767536745.83563,140334558035.00574,432978710.829921,0.02,5.772881735212298e+54,3.820613400178659e+52,5.734675601210511e+54,10.0,9713.535483340773,6.125869473095486e+25,0.37,0.009547013843082482,0.03646208164397335,0.0839341908891383,0.15014141707219703
HELP_J133205.108+375104.832,43291531282.66598,13396691551.684105,127.47410959986448,274.3080204856846,3.186414049243364e+38,5.3904978866937926e+38,4.838508989823289,1.6128363299410964,3.3,0.5,2.2190075945143195,5.563379238278754,-0.7,-0.7,2.2951036368982676e+37,2.628771065393533e+37,2.0,0.02,4.9238747022918e+37,2.6884225563080296e+38,0.47,5.0,0.0,0.0,8000.0,10.0,0.001,73897633874.50363,13.506341956499341,6.9222644280655725,13.515406306396208,10000.0,3000.0,3862.5078086081307,1.0,5.511787567655625e+36,4.60697

## Read in photoz

In [7]:
photoz=Table.read('../../dmu24/dmu24_XMM-13hr/data/XMM-13hr_DESI-DR7_Zou_et_al_2019_photo_z_withhelp_id.fits')

In [8]:
photoz

help_id,id,ra,dec,photo_z,photo_zerr
,,deg,deg,,
bytes27,bytes16,float64,float64,float64,float64
HELP_J133518.272+375026.194,6000533502003569,203.8261334268455,37.840609358038286,0.44973215,0.034033563
HELP_J133442.543+380003.910,6000534641001979,203.67726336363626,38.0010861280759,0.45776433,0.13827044
HELP_J133540.548+375155.502,6000533502003948,203.91894817791018,37.86541729173676,0.49301365,0.055531282
HELP_J133337.552+380417.817,6000534640002921,203.40646461848928,38.07161584770272,0.40855777,0.091790274
HELP_J133636.314+374336.560,6000533503001459,204.15130692615855,37.72682213743445,0.28748626,0.06836981
HELP_J133401.565+380140.968,6000534641002309,203.5065201007881,38.02804658288033,0.7450893,0.054091707
HELP_J133554.907+375940.362,6000534642001811,203.97878112024736,37.994545075037,0.6666842,0.044908088
HELP_J133437.014+380749.950,6000535777000109,203.654226891113,38.130541771185094,0.67369443,0.0627994


## Join CIGALE and photoz tables

In [9]:
prior=join(cigale,photoz)

In [10]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['photo_z']).to(u.cm))

In [11]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [12]:
prior['dec'].name='Dec'
prior['ra'].name='RA'

In [13]:
len(cigale)

3669

## Read in Maps

In [14]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/XMM-13hr_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/XMM-13hr_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/XMM-13hr_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist['PRIMARY'].header
im250hdu=hdulist['IMAGE'].header

im250=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim250=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist['IMAGE'].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist['PRIMARY'].header
im350hdu=hdulist['IMAGE'].header

im350=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim350=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist['IMAGE'].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist['PRIMARY'].header
im500hdu=hdulist['IMAGE'].header

im500=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim500=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist['IMAGE'].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [15]:
## Set XID+ prior class

In [16]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'XMM-13hr_Ldust_prediction.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'XMM-13hr_Ldust_prediction.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'XMM-13hr_Ldust_prediction',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

Private attributes "_naxis1" and "_naxis2" have been deprecated since v3.1.
Instead use the "pixel_shape" property which returns a list of NAXISj keyword values.
 [astropy.wcs.wcs]
Private attributes "_naxis1" and "_naxis2" have been deprecated since v3.1.
Instead use the "pixel_shape" property which returns a list of NAXISj keyword values.
 [astropy.wcs.wcs]


In [17]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)





##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [18]:
pixsize

array([ 6.00000015,  8.3333334 , 12.00000029])

In [19]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 70 tiles required for input catalogue and 4 large tiles
writing total_bytes=15003964...
writing bytes [0, 15003964)... done.


SystemExit: 

In [22]:
prior250.nsrc

812